In [ ]:
import os
import sys
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [ ]:
sys.path.append('../')
from src.config import CFG
from src.dataset import Img_dataset, get_transform
from src.model import Generator, Discriminator
from src.utils import show_image, conv_scale

## データセットの中身を確認

In [ ]:
# check dataset
paths = [f'{CFG.IMG_DIR}/{path}' for path in os.listdir(CFG.IMG_DIR)]
train_dataset = Img_dataset(paths, train_tran=get_transform(TRAIN=False),
                            val_tran=get_transform(TRAIN=True))

for i in range(1):
    before = train_dataset[i][0].numpy().astype(int).transpose(1, 2, 0)
    plt.imshow(before)
    plt.show()  
    

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
model_G = Generator()
model_G = nn.DataParallel(model_G)
model_G = model_G.to(device)
params_G = torch.optim.Adam(model_G.parameters(),
            lr=0.0002, betas=(0.5, 0.999))

states = torch.load(CFG.trained_param
                   )
model_G.load_state_dict(states)
model_G.eval()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [ ]:
fig, ax = plt.subplots(8, 2, figsize=(10, 80))

i = 0
for input_image, _ in train_loader:
    print(i)
    input_image = input_image.to(device)
    gen_image = model_G(input_image)
    print(input_image.shape)
    
    input_ = input_image[0].to('cpu').numpy().transpose(1, 2, 0).astype(int)
    g = conv_scale(gen_image[0].to('cpu').detach().numpy().transpose(1, 2, 0))
    ax[i][0].imshow(input_)
    ax[i][0].set_title('input')
    ax[i][1].imshow(g)
    ax[i][1].set_title('output')
    
    i += 1
    if i >= 8:
        break
    
# plt.tight_layout()
plt.show()